## Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import pickle
from tqdm import tqdm
import csv

## リストページのurlを取得

In [2]:
page_nums = range(1, 152)
listpage_urls = [f'https://imitsu.jp/ct-app-developer/search/?pn={p}#title' for p in page_nums]

## 詳細ページのurlを取得

In [3]:
url_list = []
for listpage_url in tqdm(listpage_urls):
    try:
        res = requests.get(listpage_url)
        sleep(1)
        soup = BeautifulSoup(res.text, 'html.parser')
        cards = soup.select('h3.service-title')
        urls = []
        for card in cards:
            url = card.find('a').get('href')
            urls.append(url)
        url_list.extend(urls)
    except Exception:
        print(f'Error fetching {listpage_url}')
        continue

  0%|          | 0/151 [00:00<?, ?it/s]

100%|██████████| 151/151 [05:06<00:00,  2.03s/it]


In [4]:
# Save the list of URLs to a file
with open('./data/url_list_app.pkl', 'wb') as f:
    pickle.dump(url_list, f)

## スクレイピング

In [5]:
# Load the list of URLs from the file
with open('./data/url_list_app.pkl', 'rb') as f:
    url_list = pickle.load(f)

In [6]:
with open('./data/appDev.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=['No.', '会社名', '設立年', '代表者名', '従業員数', '住所', '会社URL'])
    writer.writeheader()
    for i, url in enumerate(tqdm(url_list)):
        try:
            res = requests.get(url)
            sleep(0.3)
            soup = BeautifulSoup(res.text, 'html.parser')
            table = soup.select_one('div.supplier-information')
            dts = table.find_all('dt')
            dds = table.find_all('dd')
            name = ' - '
            year = ' - '
            rep = ' - '
            emp = ' - '
            addr = ' - '
            c_url = ' - '
            for dt, dd in zip(dts, dds):
                if dt.text == '会社名':
                    name = dd.text.replace('\n', '').strip()
                elif dt.text == '設立年':
                    year = dd.text.replace('\n', '').strip()
                elif dt.text == '代表者名':
                    rep = dd.text.replace('\n', '').strip()
                elif dt.text == '従業員数':
                    emp = dd.text.replace('\n', '').strip()
                elif dt.text == '住所':
                    addr = dd.text.replace('\n', '').strip()
                elif dt.text == '会社URL':
                    c_url = dd.text.replace('\n', '').strip()
                else:
                    continue
                item = {
                    'No.': i + 1,
                    '会社名': name,
                    '設立年': year,
                    '代表者名': rep,
                    '従業員数': emp,
                    '住所': addr,
                    '会社URL': c_url
                }
            writer.writerow(item)
        except Exception:
            print(f'Error fetching {i}, {url}')
            continue

100%|██████████| 3004/3004 [1:19:27<00:00,  1.59s/it]


## 検証

In [ ]:
result_df = pd.read_csv('./data/appDev.csv', encoding='utf-8-sig')
len(result_df)

3004